In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, GRU, CuDNNGRU, Bidirectional, Dense, InputLayer, Dropout, BatchNormalization, Reshape, Flatten, Conv1D, AveragePooling1D, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split
from util import modified_SMAPE
from tqdm import tqdm, trange, tnrange
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
demograpgics = pd.read_csv('datasets/exam-1/demographics.csv')
cc = pd.read_csv('datasets/exam-1/cc.csv')
cc.sort_values(by=['cc_no', 'pos_dt'], inplace=True)
kplus = pd.read_csv('datasets/exam-1/kplus.csv')
kplus.sort_values(by=['id', 'sunday'], inplace=True)
train_set = pd.read_csv('datasets/exam-1/train.csv')
test_set = pd.read_csv('datasets/exam-1/test.csv')

# KPLUS

In [184]:
n_transaction_threshold = 8

train_kplus = kplus.copy()
grouped = train_kplus.groupby('id')
trainable_ids = grouped.size().keys()[grouped.size() >= n_transaction_threshold]
train_kplus = train_kplus[train_kplus['id'].isin(trainable_ids)]

In [19]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
100625,2,2018-01-07,2,600
100629,2,2018-01-14,3,3200
100641,2,2018-01-21,2,600
100644,2,2018-01-28,6,3000
100643,2,2018-02-04,4,13700


In [119]:
Scaler = StandardScaler

In [120]:
scaler_x = Scaler()
train_kplus[['kp_txn_count', 'kp_txn_amt']] = scaler_x.fit_transform(train_kplus[['kp_txn_count', 'kp_txn_amt']])
train_kplus = train_kplus[train_kplus['id'] <= 50000]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [23]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt,count_amt_pca
100625,2,2018-01-07,0.002217,0.000036,-0.005816
100629,2,2018-01-14,0.004435,0.000221,-0.003707
100641,2,2018-01-21,0.002217,0.000036,-0.005816
100644,2,2018-01-28,0.011086,0.000206,0.002396
100643,2,2018-02-04,0.006652,0.000968,-0.001374


In [207]:
padding_value = float(-100)
max_len = len(kplus.groupby('sunday').groups)
sunday_id_hash = {sunday:i for i, sunday in enumerate(kplus.groupby('sunday', sort=True).groups.keys())}

id_grouped = train_kplus.groupby('id')
accept_ids = pd.Series(list(id_grouped.groups.keys()))

def create_sequence(group):
    origin = sunday_id_hash[group.iloc[0]['sunday']]

    seq = group[['kp_txn_amt', 'kp_txn_count']].to_numpy()
    pre_padding = np.ones((origin ,2), dtype=np.float32) * padding_value
    post_padding = np.ones((max_len - origin - len(seq),2), dtype=np.float32) * padding_value
    return np.concatenate((pre_padding, seq, post_padding), axis=0)
xs = np.array([create_sequence(group) for _, group in id_grouped])

ys = train_set[train_set['id'].isin(accept_ids)]['income'].to_numpy()
income_mean = train_set['income'].mean()
income_std = train_set['income'].std()
# ys = (ys - income_mean) / income_std
# ys = (ys - ys.min()) / ys.max()
scaler_y = Scaler()
ys = np.squeeze(scaler_y.fit_transform(np.expand_dims(ys,axis=2)))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [145]:
xs[-1].astype(int)

array([[  800,     1],
       [ 2700,     1],
       [  600,     1],
       [ 1000,     1],
       [ 1000,     1],
       [ 1100,     1],
       [16500,     1],
       [26800,     1],
       [25700,     2],
       [  600,     1],
       [ 1600,     1],
       [11000,     2],
       [10800,     3],
       [ 1000,     1],
       [ 6700,     2],
       [31700,     4],
       [ 3000,     1],
       [14900,     1],
       [ 7000,     1],
       [   -1,    -1],
       [   -1,    -1],
       [   -1,    -1],
       [   -1,    -1],
       [   -1,    -1],
       [   -1,    -1]])

In [80]:
ys[-1]

0.006122448979591837

In [274]:
base_activation = 'tanh'
model = Sequential()
model.add(InputLayer((25,2)))
# model.add(BatchNormalization())
# model.add(LSTM(32, return_sequences=True, ))
# model.add(Dropout(0.5))
# model.add(LSTM(32, return_sequences=True))
# model.add(Dropout(0.5))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))

model.add(Flatten())
# model.add(Dense(100))
model.add(Dense(1, activation='linear'))

In [233]:
model = Sequential()
base_activation = 'linear'
model.add(Conv1D(32, kernel_size=3, activation=base_activation, input_shape=(25,2)))
model.add(Conv1D(32, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [258]:
model = Sequential()
model.add(InputLayer((25,2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [284]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [285]:
model.compile(loss='mae', optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 25, 2)             0         
_________________________________________________________________
gru_117 (GRU)                (None, 25, 50)            7950      
_________________________________________________________________
gru_118 (GRU)                (None, 25, 50)            15150     
_________________________________________________________________
dropout_54 (Dropout)         (None, 25, 50)            0         
_________________________________________________________________
gru_119 (GRU)                (None, 25, 25)            5700      
_________________________________________________________________
gru_120 (GRU)                (None, 25, 25)            3825      
_________________________________________________________________
dropout_55 (Dropout)         (None, 25, 25)            0         
__________

In [267]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [54]:
class EvaluateSMAPE(Callback):
    def __init__(self, xs, ys_true, name, scaler_y=None):
        self.scaler_y = scaler_y
        self.xs = xs
        
        self.ys_true = np.squeeze(scaler_y.inverse_transform(ys_true))
        self.name = name
    def on_epoch_end(self, epoch, logs=None):
        ys_pred = model.predict(self.xs)
        if self.scaler_y is not None:
            ys_pred = np.squeeze(self.scaler_y.inverse_transform(ys_pred))
        score = modified_SMAPE(self.ys_true, ys_pred)
        logs[f'{self.name}-SMAPE'] = score
        print(f'Epoch {epoch+1} | {self.name}-SMAPE: {score}')
        

In [286]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=scaler_y),
                     TensorBoard(log_dir='logs/gruV4')],
          initial_epoch=49,
         epochs=10000)

Train on 27725 samples, validate on 6932 samples
Epoch 50/10000
27725/27725 [==============================] - 43s 2ms/step - loss: 0.4070 - val_loss: 0.4571
Epoch 50 | val-SMAPE: 87.15221654180186
Epoch 51/10000
27725/27725 [==============================] - 38s 1ms/step - loss: 0.4047 - val_loss: 0.4472
Epoch 51 | val-SMAPE: 89.1394895230774
Epoch 52/10000
27725/27725 [==============================] - 38s 1ms/step - loss: 0.4028 - val_loss: 0.4526
Epoch 52 | val-SMAPE: 88.67333321926806
Epoch 53/10000
27725/27725 [==============================] - 38s 1ms/step - loss: 0.4032 - val_loss: 0.4566
Epoch 53 | val-SMAPE: 87.59214888186834
Epoch 54/10000
27725/27725 [==============================] - 38s 1ms/step - loss: 0.4014 - val_loss: 0.4523
Epoch 54 | val-SMAPE: 87.71205892823158
Epoch 55/10000
27725/27725 [==============================] - 38s 1ms/step - loss: 0.3992 - val_loss: 0.4579
Epoch 55 | val-SMAPE: 87.86274694079495
Epoch 56/10000
27725/27725 [==============================

KeyboardInterrupt: 

In [278]:
model.save('weight/gruV4-SMAPE:87.7981.h5')

# Credit

In [3]:
cc_id = pd.merge(cc, demograpgics[['id','cc_no']], on='cc_no')

In [4]:
cc_id['count'] = 1

In [5]:
cc_id.head(10)

,cc_no,pos_dt,cc_txn_amt,id,count
0,2,2018-03-10,800,2,1
1,2,2018-03-12,3800,2,1
2,2,2018-04-27,14700,2,1
3,2,2018-04-29,4000,2,1
4,2,2018-05-07,800,2,1
5,2,2018-05-14,800,2,1
6,2,2018-06-04,1000,2,1
7,2,2018-06-11,1000,2,1
8,4,2018-05-11,20000,4,1
9,4,2018-05-11,30000,4,1


In [6]:
grouped = cc_id.groupby(['id', 'pos_dt'])

In [7]:
cc_persons = grouped.sum().reset_index()
cc_persons.head(10)

,id,pos_dt,cc_no,cc_txn_amt,count
0,1,2018-01-20,98397,4700,1
1,1,2018-02-17,196794,20000,2
2,1,2018-05-13,98397,10000,1
3,1,2018-06-14,98397,10000,1
4,2,2018-01-04,9740,1600,1
5,2,2018-01-28,29220,3500,3
6,2,2018-01-29,9740,1500,1
7,2,2018-02-04,9740,1100,1
8,2,2018-02-11,9740,800,1
9,2,2018-02-19,9740,800,1


In [8]:
Scaler = StandardScaler
cc_scaler = Scaler()
train_scaler = Scaler()
cc_persons[['cc_txn_amt', 'count']] = cc_scaler.fit_transform(cc_persons[['cc_txn_amt', 'count']])
scaled_train_set = train_set.copy()
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'].to_numpy(), axis=1))

/home/porlolicon/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/porlolicon/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/porlolicon/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/porlolicon/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
cc_persons.head()

,id,pos_dt,cc_no,cc_txn_amt,count
0,1,2018-01-20,98397,0.009926,-0.366991
1,1,2018-02-17,196794,0.421319,1.115008
2,1,2018-05-13,98397,0.152435,-0.366991
3,1,2018-06-14,98397,0.152435,-0.366991
4,2,2018-01-04,9740,-0.073428,-0.366991


In [10]:
scaled_train_set.head()

,id,income
0,1,-0.525099
1,2,1.090529
2,3,-0.356022
3,4,0.245142
4,5,-0.562672


In [11]:
cc_persons = cc_persons[cc_persons['id'] <= 50000]

In [12]:
origin_timestamp = pd.Timestamp(cc['pos_dt'].min())
max_len_seq = (pd.Timestamp(cc['pos_dt'].max()) - origin_timestamp).days + 1
cc_persons['pos_dt_index'] = cc_persons.apply(lambda row: (pd.Timestamp(row['pos_dt']) - origin_timestamp).days, axis=1)

In [13]:
padding_value = float(-100)
xs = [] # [[cc_txn_amt, count], ...]
ys = []
train_id_set = scaled_train_set.set_index('id')
for _id, group in cc_persons.groupby('id'):
    seq = np.ones((max_len_seq, 2), dtype=np.float32) * padding_value
    seq[group['pos_dt_index'].to_numpy()] = group[['cc_txn_amt', 'count']].to_numpy()
    
    xs.append(seq)
    ys.append(train_id_set.loc[_id]['income'])
xs = np.asarray(xs)
ys = np.asarray(ys)

In [216]:
xs

array([[[-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        ...,
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02]],

       [[-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        ...,
        [-6.8050250e-02, -3.6699140e-01],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.4273417e-02,  1.1150075e+00]],

       [[-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        ...,
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02]],

       ...,

       [[-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02],
        ...,
        [-9.4938666e-02, -3.6699140e-01],
     

In [215]:
ys

array([-0.52509903,  1.09052903,  0.24514225, ..., -0.2057307 ,
       -0.54388541, -0.65660364])

In [48]:
base_activation = 'tanh'
model = Sequential()
model.add(InputLayer((max_len_seq,2)))
model.add(Bidirectional(CuDNNGRU(128, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(CuDNNGRU(128, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(CuDNNGRU(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(CuDNNGRU(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(CuDNNGRU(32, )))
model.add(Activation(base_activation))
# model.add(Flatten())
# model.add(Dense(100))
model.add(Dense(1, activation='linear'))

In [45]:
model = Sequential()
base_activation = 'relu'
model.add(Conv1D(32, kernel_size=3, activation=base_activation, input_shape=(max_len_seq,2)))
model.add(Conv1D(32, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [63]:
model = Sequential()
model.add(InputLayer((max_len_seq,2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [49]:
def SMAPE_loss(scaler):
    mean = K.constant(scaler.mean_)
    std = K.constant(np.sqrt(scaler.var_))
    def loss_func(y_true, y_pred):
        y_true = (y_true * std) + mean
        y_pred = (y_pred * std) + mean
        return K.mean((K.abs(y_pred - y_true) ** 2) / (( K.minimum(K.abs(y_true)*2, K.abs(y_pred)) + K.abs(y_true)) ** 2)) 
    return loss_func

In [30]:
def SMAPE_np(y_true, y_pred):
    a , f = np.array(y_true) ,np.array(y_pred)
    return 100 - (100) * np.mean(np.power(np.abs(f - a),2) / np.power(np.minimum(2 * np.abs(a),np.abs(f)) + np.abs(a),2))


In [50]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [51]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=opt )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 181, 256)          101376    
_________________________________________________________________
activation_1 (Activation)    (None, 181, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 181, 256)          296448    
_________________________________________________________________
activation_2 (Activation)    (None, 181, 256)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 181, 256)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 181, 128)          123648    
_________________________________________________________________
activation_3 (Activation)    (None, 181, 128)         

In [52]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [55]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler),
                     TensorBoard(log_dir='logs/cc_gru_smape')],
          initial_epoch=0,
         epochs=10000)

W1028 20:19:17.997876 140082931795776 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1028 20:19:19.708583 140082931795776 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W1028 20:19:19.714822 140082931795776 module_wrapper.py:137] From /home/porlolicon/.local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



Train on 31960 samples, validate on 7990 samples
Epoch 1/10000
31960/31960 [==============================] - 68s 2ms/step - loss: 0.1091 - val_loss: 0.0943
Epoch 1 | val-SMAPE: 90.57426353704828
Epoch 2/10000
31960/31960 [==============================] - 65s 2ms/step - loss: 0.0971 - val_loss: 0.0969
Epoch 2 | val-SMAPE: 90.30955666196212
Epoch 3/10000
31960/31960 [==============================] - 66s 2ms/step - loss: 0.0961 - val_loss: 0.0934
Epoch 3 | val-SMAPE: 90.65647471594993
Epoch 4/10000
31960/31960 [==============================] - 66s 2ms/step - loss: 0.0964 - val_loss: 0.1013
Epoch 4 | val-SMAPE: 89.86831715661117
Epoch 5/10000
10688/31960 [=========>....................] - ETA: 39s - loss: 0.0980

KeyboardInterrupt: 

60.21697

In [56]:
train_scaler.inverse_transform(y_val[:10])

array([75000., 37000., 17000., 43000., 50000., 23000., 50000., 14000.,
       88000., 22000.])

In [59]:
np.squeeze(train_scaler.inverse_transform(model.predict(x_val[:10]))).astype(np.int)

array([36747, 23314, 21383, 23101, 47924, 31014, 41069, 29239, 35962,
       22235])

In [60]:
incomes = train_scaler.inverse_transform(y_val[:1000])

In [65]:
modified_SMAPE(incomes, incomes)

100.0

In [66]:
modified_SMAPE(incomes, incomes - 10000)

90.45559042413171

In [67]:
modified_SMAPE(incomes, incomes + 10000)

97.37249598212075